# AlgTest Process TPM 2.0 data visualisation

This is a test for visualisation of html outputs in jupyter notebooks

## 1. Results
First we need to get TPM 2.0 results. Upload a folder with results in the left corner. Make sure your results are zipped and zip has following file structure. Following structure shows possible dir structures of the zip file:

```
tpmalgtest_results - This folder must be created when extracting
│
│
└───<tpm name>
│   │───detail
│   │   │─── docker_info.csv
│   │   │─── Keygen_*.csv
│   │   │─── keygen_*.csv
│   │   └─── Quicktest_*.csv
│   │───performance
│   │   └─── <tpm name>.csv -> Performance result - Important
│   └───results
│       └─── <tpm name>.csv -> Support result - Important
│─── ...
│
│───<tpm name>
│   └───out
│       └───detail
│       │    │─── docker_info.csv
│       │    │─── Keygen_*.csv
│       │    │─── keygen_*.csv
│       │    └─── Quicktest_*.csv
│       │───performance
│       │   └─── <tpm name>.csv -> Performance result - Important
│       └───results
│           └─── <tpm name>.csv -> Support result - Important
│─── ...
│   
└───out.*
│   │─── docker_info.csv
│   │─── Keygen_*.csv
│   │─── keygen_*.csv
│   └─── Quicktest_*.csv
└─── ...
```

### a) Unzipping

In [1]:
import zipfile
# Fill in the filename
FILENAME = "tpmalgtest_results.zip"

with zipfile.ZipFile(FILENAME,"r") as zip_ref:
    zip_ref.extractall(".")

### b) Parsing results

In [2]:
import os
from algtestprocess.modules.parser.tpm.performance import PerformanceParserTPM
from algtestprocess.modules.parser.tpm.support import SupportParserTPM

def get_tpm_profiles():
    # First we get all the files in the dir
    files =[os.path.join(root, file) for root, dirs, files in os.walk("./tpmalgtest_results") for file in files]
    # Then we filter performance and support result .csv files
    performance = list(filter(lambda name: "/performance/" in name and ".csv" in name, files))
    support =  list(filter(lambda name: "/results/" in name and ".csv" in name, files))
    return performance, support

performance_files, support_files = get_tpm_profiles()

# Then we parse each profile with its respective parser
performance_profiles = list(map(lambda file: PerformanceParserTPM(file).parse(), performance_files))
support_profiles = list(map(lambda file: SupportParserTPM(file).parse(), support_files))

# We can filter out profiles without results

performance_profiles = list(filter(lambda profile: profile.results, performance_profiles))
support_profiles = list(filter(lambda profile: profile.results, support_profiles))


### c) Utility functions

In [3]:
def choose(data: list[tuple[str, str]]):
    print("Please choose from these visualisations or write -1 to end")
    for idx, item in enumerate(data):
        print(f"{idx} - {item[0]}")
    
    which =  int(input())
    
    if which == -1:
        return
    
    if which < 0 or which >= len(data):
        print(f"Wrong choice, please choose correctly from range 0 - {len(data)-1} or -1 to end")
        choose(data)
        return
    
    print(f"Chosen {data[which]}")
    
    return IFrame(data[which][1], width=1400, height=900)

## 2. Visualisations

### a) Support table

In [4]:
from algtestprocess.modules.pages.support import SupportTPM
from IPython.display import IFrame, display, HTML

# Now we generate the file
# notebook argument means that the function will inline all needed js / css
# output path makes the function actually write the html into file
html, path = SupportTPM(support_profiles).run(notebook=True, output_path=".")

# We can view that file using IFrame or manually open it in browser
IFrame(path, width=1400, height=900)

### b) Similarity table

In [5]:
from algtestprocess.modules.pages.similarity import SimilarityTPM

html, path = SimilarityTPM(performance_profiles).run(notebook=True, output_path=".")

IFrame(path, width=1400, height=900)

### c) Radar graphs

In [6]:
from algtestprocess.modules.pages.radar import RadarTPM

# For visualisations which create multiple htmls return list of tuples (name, path)
_, data = RadarTPM(performance_profiles).run(notebook=True, output_path=".")

# Uncomment to visualise radar graphs for TPMs
#choose(data)

### d) Execution time

In [8]:
from algtestprocess.modules.pages.executiontime import ExecutionTimeTPM

_, data = ExecutionTimeTPM(performance_profiles).run(notebook=True, output_path=".")

# Uncomment to visualise radar graphs for TPMs
#choose(data)